# DNN Critic model

In [1]:
import pandas as pd 
import numpy as np  
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
from collections import Counter
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras import optimizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# read data
mydt = pd.read_csv("../dataset/KDD-2018/trainu2r.csv")
print(mydt['Label'].value_counts())

# Create matrix of features and matrix of target variable 
dataset = mydt.values
X = dataset[:,0:41]
y = dataset[:,41]

# convert the scaled array to dataframe 
min_max_scaler = MinMaxScaler()
X_scale = min_max_scaler.fit_transform(X)

X_full = X_scale
y_full = y

X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, test_size=0.3, random_state=1)

print(Counter(y_train))
print(Counter(y_test))

# define the keras model
model = Sequential()
model.add(Dense(128, input_shape=(41,), activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(41, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print("Building Critic WGAN model...")
# fit the keras model on the dataset
mod = model.fit(X_train, y_train, epochs=40, batch_size=100, verbose=1, validation_data=(X_test, y_test))

# make class predictions with the model
y_pred = (model.predict(X_test) > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100))

# Save model
# serialize model to JSON
model_json = model.to_json()
with open("DNN_critic_model_KDD_3.json", "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
model.save_weights("DNN_critic_model_KDD_3.h5")
print("Saved model!")

0    14019
1       24
Name: Label, dtype: int64
Counter({0.0: 9810, 1.0: 20})
Counter({0.0: 4209, 1.0: 4})


2022-09-04 15:50:36.895963: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Building Critic WGAN model...
Epoch 1/40
99/99 [==============================] - 2s 12ms/step - loss: 0.0789 - accuracy: 0.9858 - val_loss: 0.0066 - val_accuracy: 0.9991
Epoch 2/40
99/99 [==============================] - 1s 7ms/step - loss: 0.0112 - accuracy: 0.9980 - val_loss: 0.0041 - val_accuracy: 0.9991
Epoch 3/40
99/99 [==============================] - 1s 8ms/step - loss: 0.0075 - accuracy: 0.9981 - val_loss: 0.0021 - val_accuracy: 0.9991
Epoch 4/40
99/99 [==============================] - 1s 6ms/step - loss: 0.0049 - accuracy: 0.9988 - val_loss: 0.0011 - val_accuracy: 0.9998
Epoch 5/40
99/99 [==============================] - 1s 7ms/step - loss: 0.0036 - accuracy: 0.9993 - val_loss: 0.0011 - val_accuracy: 0.9998
Epoch 6/40
99/99 [==============================] - 1s 6ms/step - loss: 0.0032 - accuracy: 0.9994 - val_loss: 0.0014 - val_accuracy: 0.9993
Epoch 7/40
99/99 [==============================] - 1s 6ms/step - loss: 0.0030 - accuracy: 0.9995 - val_loss: 0.0013 - val_accura

In [2]:
# Testing 
cm = confusion_matrix(y_test, y_pred) 
tnr = recall_score(y_test, y_pred) 
fpr = 1 - tnr

print ("confusion_matrix for test data\n",cm)
print(classification_report(y_test,y_pred,labels=np.unique(y_pred)))
print('Accuracy: %.4f' % accuracy_score(y_test, y_pred))
print('Precision: %.4f' % precision_score(y_test, y_pred))
print('Recall: %.4f' % recall_score(y_test, y_pred))
print('F1 Score: %.4f' % f1_score(y_test, y_pred))
print('FPR Score: %.4f' % fpr)

confusion_matrix for test data
 [[4204    5]
 [   0    4]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4209
           1       0.44      1.00      0.62         4

    accuracy                           1.00      4213
   macro avg       0.72      1.00      0.81      4213
weighted avg       1.00      1.00      1.00      4213

Accuracy: 0.9988
Precision: 0.4444
Recall: 1.0000
F1 Score: 0.6154
FPR Score: 0.0000


# WGAN

In [3]:
import torch 
import torch.nn
import torch.nn.functional as nn 
import torch.autograd as autograd 
import torch.optim as optim 
import torch.utils.data as Data
from torch.autograd import Variable
import numpy as np 
import tensorflow as tf 
import os
import pandas as pd
from sklearn.preprocessing import MinMaxScaler 
from sklearn.metrics import confusion_matrix 
import keras
from keras.models import Sequential 
from keras.layers import Dense 
from keras import optimizers
from keras.models import model_from_json
import matplotlib 
matplotlib.use('agg') 
import matplotlib.pyplot as plt 
import matplotlib.gridspec as gridspec 

# hyperparameters 
mb_size = 64 
z_dim = 100 
X_dim = 41 
y_dim = 1 
h_dim = 128 #hidden 
lr = 0.00005 #learning_rate

# read data
df = pd.read_csv("../dataset/KDD-2018/trainu2r.csv")
print('original data label features number:')
print(df['Label'].value_counts())

# data preparation
criteria0 = df['Label'] == 1 #filter attack data 
data0 = df[criteria0]

float_array = data0.values.astype(float) 
scaler = MinMaxScaler()
scaled_array = scaler.fit_transform(float_array)
attack_data = pd.DataFrame(scaled_array)
attack_data.columns = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes',
       'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot',
       'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell',
       'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
       'num_access_files', 'num_outbound_cmds', 'is_host_login',
       'is_guest_login', 'count', 'srv_count', 'serror_rate',
       'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
       'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
       'dst_host_srv_count', 'dst_host_same_srv_rate',
       'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
       'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
       'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
       'dst_host_srv_rerror_rate', 'Label']

print('----------------------------------------') 
print('attack value number:') 
print(attack_data['Label'].value_counts())

# transform data to tensor format
train_target = torch.tensor(attack_data['Label'].values.astype(np.float32)) 
train = torch.tensor(attack_data.drop(['Label'], axis = 1).values.astype(np.float32)) 
train_tensor = Data.TensorDataset(train, train_target) 

#creat DataLoader with batch and shuffle
train_loader = torch.utils.data.DataLoader(dataset = train_tensor, batch_size = mb_size, shuffle = True)

# define models
# Generator
G = torch.nn.Sequential(
    torch.nn.Linear(z_dim, h_dim), 
    torch.nn.ReLU(),
    torch.nn.Linear(h_dim, h_dim), 
    torch.nn.ReLU(),
    torch.nn.Linear(h_dim, X_dim), 
    torch.nn.Sigmoid()
)

# Discriminator
D = torch.nn.Sequential( 
    torch.nn.Linear(X_dim, h_dim), 
    torch.nn.ReLU(),
    torch.nn.Linear(h_dim, h_dim), 
    torch.nn.ReLU(),
    # No sigmoid
    torch.nn.Linear(h_dim, 1),
)

# setup cpu using cpu (if use gpu -> set cpu to cuda)
G.cpu()
D.cpu()

def reset_grad():
    G.zero_grad()
    D.zero_grad()
    
#initialize optimizers and dataholders
G_solver = optim.RMSprop(G.parameters(), lr=lr)
D_solver = optim.RMSprop(D.parameters(), lr=lr)
G_losses = []
D_losses = []
data_iter = iter(train_loader)

original data label features number:
0    14019
1       24
Name: Label, dtype: int64
----------------------------------------
attack value number:
0.0    24
Name: Label, dtype: int64


In [4]:
#training 
print("Start training WGAN model...")
d_iter = 2 
for it in range(500000):
    if it<100: d_iter = 50 
    else:
        d_iter = 2
    for _ in range(d_iter):
        # Sample data
        z = Variable(torch.randn(mb_size, z_dim)).cpu()
        try:
            X, _ = data_iter.next() 
        except StopIteration:
            data_iter = iter(train_loader) 
            X, _ = data_iter.next()
        #X = Variable(torch.from_numpy(X))
        X = X.cpu()
        
        # Dicriminator forward-loss-backward-update 
        G_sample = G(z)
        D_real = D(X)
        D_fake = D(G_sample)
        
        D_loss = -(torch.mean(D_real) - torch.mean(D_fake))
        
        D_loss.backward() 
        D_solver.step()
        
        # Weight clipping 
        for p in D.parameters():
            p.data.clamp_(-0.01, 0.01)
            
        D_losses.append(D_loss.data.cpu().numpy())
        
        # Housekeeping - reset gradient 
        reset_grad()
        
    # Generator forward-loss-backward-update 
    #X = Variable(torch.from_numpy(X)) 
    z = Variable(torch.randn(mb_size, z_dim)).cpu() #change cpu to cuda
    
    G_sample = G(z)
    D_fake = D(G_sample)
    
    G_loss = -torch.mean(D_fake)
    
    G_loss.backward() 
    G_solver.step()

    # Housekeeping - reset gradient 
    reset_grad()

    G_losses.append(G_loss.data.cpu().numpy())

    # Print and plot every now and then 
    if it % 1000 == 0:
        print('Iter-{}; D_loss: {}; G_loss: {}'
              .format(it, D_loss.data.cpu().numpy(),  
                G_loss.data.cpu().numpy()))
        
        samples = G(z).data.numpy()[:16]

#plot losses
plt.figure(figsize=(10,5))
plt.title("Generator and Critic Loss During Training") 
plt.plot(G_losses,label="Generator") 
plt.plot(D_losses,label="Critic") 
plt.xlabel("iter")
plt.ylabel("loss")
plt.legend()
plt.savefig('KDD_loss_3.png')

#save model
torch.save({'modelG_state_dict': G.state_dict(), 
            'modelD_state_dict': D.state_dict(),
            'optimizerG_state_dict': G_solver.state_dict(),
            'optimizerD_state_dict': D_solver.state_dict(),
            'G_loss': G_losses,
            'D_loss': D_losses
           }, "WGAN_model_KDD_3.pth")

Start training WGAN model...
Iter-0; D_loss: -0.005068625323474407; G_loss: 0.013880228623747826
Iter-1000; D_loss: -0.00633380189538002; G_loss: 0.017341773957014084
Iter-2000; D_loss: -0.004150146618485451; G_loss: 0.015435497276484966
Iter-3000; D_loss: -0.0023359842598438263; G_loss: 0.015531826764345169
Iter-4000; D_loss: -0.0034338347613811493; G_loss: 0.01618196628987789
Iter-5000; D_loss: -0.003324568271636963; G_loss: 0.01457901205867529
Iter-6000; D_loss: -0.0030789943411946297; G_loss: 0.01398596353828907
Iter-7000; D_loss: -0.002639944665133953; G_loss: 0.013653662987053394
Iter-8000; D_loss: -0.0025704773142933846; G_loss: 0.014607190154492855
Iter-9000; D_loss: -0.0025943145155906677; G_loss: 0.015717001631855965
Iter-10000; D_loss: -0.002724352292716503; G_loss: 0.016505513340234756
Iter-11000; D_loss: -0.002613503485918045; G_loss: 0.016129761934280396
Iter-12000; D_loss: -0.002214341424405575; G_loss: 0.015606915578246117
Iter-13000; D_loss: -0.002608422189950943; G_lo

Iter-113000; D_loss: -0.001249796710908413; G_loss: 0.014955630525946617
Iter-114000; D_loss: -0.001206839457154274; G_loss: 0.014267386868596077
Iter-115000; D_loss: -0.0011966628953814507; G_loss: 0.013949678279459476
Iter-116000; D_loss: -0.001180192455649376; G_loss: 0.014373449608683586
Iter-117000; D_loss: -0.0012033246457576752; G_loss: 0.014237412251532078
Iter-118000; D_loss: -0.0011919951066374779; G_loss: 0.013869471848011017
Iter-119000; D_loss: -0.0011627161875367165; G_loss: 0.014204798266291618
Iter-120000; D_loss: -0.0012272028252482414; G_loss: 0.014167312532663345
Iter-121000; D_loss: -0.001203363761305809; G_loss: 0.014775536954402924
Iter-122000; D_loss: -0.0011409372091293335; G_loss: 0.014132466167211533
Iter-123000; D_loss: -0.0011620856821537018; G_loss: 0.014215493574738503
Iter-124000; D_loss: -0.0011204220354557037; G_loss: 0.01432544831186533
Iter-125000; D_loss: -0.0011616889387369156; G_loss: 0.014352953992784023
Iter-126000; D_loss: -0.0012155268341302872

Iter-225000; D_loss: -0.0010884860530495644; G_loss: 0.01384445559233427
Iter-226000; D_loss: -0.0011492818593978882; G_loss: 0.013775166124105453
Iter-227000; D_loss: -0.001116645522415638; G_loss: 0.01362032350152731
Iter-228000; D_loss: -0.0012221187353134155; G_loss: 0.013716284185647964
Iter-229000; D_loss: -0.0011432813480496407; G_loss: 0.01465962827205658
Iter-230000; D_loss: -0.0011678989976644516; G_loss: 0.01436951756477356
Iter-231000; D_loss: -0.0011577391996979713; G_loss: 0.013886451721191406
Iter-232000; D_loss: -0.0010935906320810318; G_loss: 0.014544411562383175
Iter-233000; D_loss: -0.0011105714365839958; G_loss: 0.01371635403484106
Iter-234000; D_loss: -0.0011867545545101166; G_loss: 0.013409673236310482
Iter-235000; D_loss: -0.0011287899687886238; G_loss: 0.014420729130506516
Iter-236000; D_loss: -0.0011798515915870667; G_loss: 0.014451224356889725
Iter-237000; D_loss: -0.0012035630643367767; G_loss: 0.013798169791698456
Iter-238000; D_loss: -0.001142469234764576; 

Iter-337000; D_loss: -0.0011352291330695152; G_loss: 0.014302454888820648
Iter-338000; D_loss: -0.0010995641350746155; G_loss: 0.014428772032260895
Iter-339000; D_loss: -0.001168704591691494; G_loss: 0.01422105636447668
Iter-340000; D_loss: -0.001188153401017189; G_loss: 0.014314880594611168
Iter-341000; D_loss: -0.0010986169800162315; G_loss: 0.014232000336050987
Iter-342000; D_loss: -0.001242196187376976; G_loss: 0.014229916036128998
Iter-343000; D_loss: -0.0011871680617332458; G_loss: 0.013795865699648857
Iter-344000; D_loss: -0.0012203315272927284; G_loss: 0.014487804844975471
Iter-345000; D_loss: -0.001255837269127369; G_loss: 0.01403825730085373
Iter-346000; D_loss: -0.0011224588379263878; G_loss: 0.014340803027153015
Iter-347000; D_loss: -0.0012066168710589409; G_loss: 0.014827720820903778
Iter-348000; D_loss: -0.0011142129078507423; G_loss: 0.014399484731256962
Iter-349000; D_loss: -0.001090395264327526; G_loss: 0.014283413998782635
Iter-350000; D_loss: -0.0011189570650458336; 

Iter-449000; D_loss: -0.0011658454313874245; G_loss: 0.014388644136488438
Iter-450000; D_loss: -0.0011535407975316048; G_loss: 0.0143004534766078
Iter-451000; D_loss: -0.001180681400001049; G_loss: 0.014587626792490482
Iter-452000; D_loss: -0.0011883731931447983; G_loss: 0.0143692372366786
Iter-453000; D_loss: -0.001147586852312088; G_loss: 0.014199459925293922
Iter-454000; D_loss: -0.0011326679959893227; G_loss: 0.013889783062040806
Iter-455000; D_loss: -0.0011235512793064117; G_loss: 0.014387261122465134
Iter-456000; D_loss: -0.0011285636574029922; G_loss: 0.014118392020463943
Iter-457000; D_loss: -0.0011198772117495537; G_loss: 0.014002351090312004
Iter-458000; D_loss: -0.0011764680966734886; G_loss: 0.013896418735384941
Iter-459000; D_loss: -0.0012141596525907516; G_loss: 0.013959166593849659
Iter-460000; D_loss: -0.0010999999940395355; G_loss: 0.014500541612505913
Iter-461000; D_loss: -0.0011606914922595024; G_loss: 0.01418771967291832
Iter-462000; D_loss: -0.0012012748047709465; 

# Data generation from saved WGAN model

In [5]:
#loaded NN model
print("Loading model...")
print("----------------------------------------")
json_file = open("DNN_critic_model_KDD_3.json","r") 
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json) 
loaded_model.load_weights("DNN_critic_model_KDD_3.h5")

G = torch.nn.Sequential(
    torch.nn.Linear(z_dim, h_dim),
    torch.nn.ReLU(),
    torch.nn.Linear(h_dim, h_dim),
    torch.nn.ReLU(),
    torch.nn.Linear(h_dim, X_dim), 
    torch.nn.Sigmoid()
)

checkpoint = torch.load("WGAN_model_KDD_3.pth")
G.load_state_dict(checkpoint['modelG_state_dict'])

G.eval()
G.cpu()

final_Data = pd.DataFrame(columns = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes',
       'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot',
       'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell',
       'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
       'num_access_files', 'num_outbound_cmds', 'is_host_login',
       'is_guest_login', 'count', 'srv_count', 'serror_rate',
       'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
       'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
       'dst_host_srv_count', 'dst_host_same_srv_rate',
       'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
       'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
       'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
       'dst_host_srv_rerror_rate', 'Label']) 
print("empty df",final_Data) 
print("----------------------------------------")
print("Completed loading model")

Loading model...
----------------------------------------
empty df Empty DataFrame
Columns: [duration, protocol_type, service, flag, src_bytes, dst_bytes, land, wrong_fragment, urgent, hot, num_failed_logins, logged_in, num_compromised, root_shell, su_attempted, num_root, num_file_creations, num_shells, num_access_files, num_outbound_cmds, is_host_login, is_guest_login, count, srv_count, serror_rate, srv_serror_rate, rerror_rate, srv_rerror_rate, same_srv_rate, diff_srv_rate, srv_diff_host_rate, dst_host_count, dst_host_srv_count, dst_host_same_srv_rate, dst_host_diff_srv_rate, dst_host_same_src_port_rate, dst_host_srv_diff_host_rate, dst_host_serror_rate, dst_host_srv_serror_rate, dst_host_rerror_rate, dst_host_srv_rerror_rate, Label]
Index: []

[0 rows x 42 columns]
----------------------------------------
Completed loading model


In [6]:
it =0

while(final_Data.shape[0]<20000):
    it+=1
    z = Variable(torch.randn(100, z_dim)).cpu() 
    with torch.no_grad():
        data = G(z)
    data = data.cpu().numpy() 
    fake = pd.DataFrame(data)
        
    newy = np.ones((fake.shape[0],)) 
    #this is the model which changes 
    new_pred = loaded_model.predict(fake)
    new_pred = (new_pred > 0.5)
    cm = confusion_matrix(newy,new_pred)
    print("-------------------------------------------------------------") 
    print (" confusion_matrix for new data with deeper model \n" ,cm)
    
    combined_data = np.concatenate((fake,new_pred),axis=1) 
    combined_data = pd.DataFrame(combined_data)
    combined_data.columns = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes',
       'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot',
       'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell',
       'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
       'num_access_files', 'num_outbound_cmds', 'is_host_login',
       'is_guest_login', 'count', 'srv_count', 'serror_rate',
       'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
       'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
       'dst_host_srv_count', 'dst_host_same_srv_rate',
       'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
       'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
       'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
       'dst_host_srv_rerror_rate', 'Label']
    
    criteria1 = combined_data['Label'] == 1 #just attack data 
    data1 = combined_data[criteria1]
    print("only attack data - ",data1.shape) 
    final_Data = final_Data.append(data1, ignore_index = True) 
    print("final data ", it, "- ",final_Data.shape) 
    print("-------------------------------------------------------------") 
    
print(final_Data['Label'].value_counts())
print("---------------------------------------------------------")
df2 = final_Data.sample(n=13995, replace=False, random_state=1)
df3 = final_Data.sample(n=20000, replace=False, random_state=1)
final = pd.concat([mydt,df2], ignore_index = True,axis=0)
final['Label'].value_counts()
df3.to_csv("../dataset/KDD-2018/generated_u2r.csv", index = False, header = True)
final.to_csv("../dataset/KDD-2018/balanced_data_u2r.csv", index = False, header = True)

4/4 [==============================] - 0s 5ms/step
-------------------------------------------------------------
 confusion_matrix for new data with deeper model 
 [[ 0  0]
 [67 33]]
only attack data -  (33, 42)
final data  1 -  (33, 42)
-------------------------------------------------------------
4/4 [==============================] - 0s 3ms/step
-------------------------------------------------------------
 confusion_matrix for new data with deeper model 
 [[ 0  0]
 [66 34]]
only attack data -  (34, 42)
final data  2 -  (67, 42)
-------------------------------------------------------------


/tmp/ipykernel_3245245/1638766432.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_Data = final_Data.append(data1, ignore_index = True)
/tmp/ipykernel_3245245/1638766432.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_Data = final_Data.append(data1, ignore_index = True)


4/4 [==============================] - 0s 3ms/step
-------------------------------------------------------------
 confusion_matrix for new data with deeper model 
 [[ 0  0]
 [62 38]]
only attack data -  (38, 42)
final data  3 -  (105, 42)
-------------------------------------------------------------
4/4 [==============================] - 0s 3ms/step
-------------------------------------------------------------
 confusion_matrix for new data with deeper model 
 [[ 0  0]
 [70 30]]
only attack data -  (30, 42)
final data  4 -  (135, 42)
-------------------------------------------------------------


/tmp/ipykernel_3245245/1638766432.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_Data = final_Data.append(data1, ignore_index = True)
/tmp/ipykernel_3245245/1638766432.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_Data = final_Data.append(data1, ignore_index = True)


4/4 [==============================] - 0s 3ms/step
-------------------------------------------------------------
 confusion_matrix for new data with deeper model 
 [[ 0  0]
 [69 31]]
only attack data -  (31, 42)
final data  5 -  (166, 42)
-------------------------------------------------------------
4/4 [==============================] - 0s 3ms/step
-------------------------------------------------------------
 confusion_matrix for new data with deeper model 
 [[ 0  0]
 [73 27]]
only attack data -  (27, 42)
final data  6 -  (193, 42)
-------------------------------------------------------------


...

/tmp/ipykernel_3245245/1638766432.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_Data = final_Data.append(data1, ignore_index = True)
/tmp/ipykernel_3245245/1638766432.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_Data = final_Data.append(data1, ignore_index = True)


4/4 [==============================] - 0s 3ms/step
-------------------------------------------------------------
 confusion_matrix for new data with deeper model 
 [[ 0  0]
 [74 26]]
only attack data -  (26, 42)
final data  597 -  (19965, 42)
-------------------------------------------------------------
4/4 [==============================] - 0s 3ms/step


/tmp/ipykernel_3245245/1638766432.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_Data = final_Data.append(data1, ignore_index = True)


-------------------------------------------------------------
 confusion_matrix for new data with deeper model 
 [[ 0  0]
 [70 30]]
only attack data -  (30, 42)
final data  598 -  (19995, 42)
-------------------------------------------------------------
4/4 [==============================] - 0s 2ms/step
-------------------------------------------------------------
 confusion_matrix for new data with deeper model 
 [[ 0  0]
 [60 40]]
only attack data -  (40, 42)
final data  599 -  (20035, 42)
-------------------------------------------------------------
1.0    20035
Name: Label, dtype: int64
---------------------------------------------------------


/tmp/ipykernel_3245245/1638766432.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_Data = final_Data.append(data1, ignore_index = True)
/tmp/ipykernel_3245245/1638766432.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_Data = final_Data.append(data1, ignore_index = True)
